<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=725bb00533fa1b7d6d4240012e7f3d18051f32d3a06bfa461e6e09644e86bc5d
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-11 15:23:11
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.40 C
-------------------
Today PnL: 60.09 K (0.43%)
Current PnL: -28.69 L (-18.53%)
CY Booked + Current PnL: -14.40 L (-9.3%)
-------------------
Total profit:  1.28 L
Total loss:  -29.97 L
-------------------
Total Booked + Current PnL: 12.58 L (9.84%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.24%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 93.32 L (66.85%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 4.93%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,0.83,-16.72,20.18,0.09,16937.0,-16845.0,83932.0,94.88,43.0,M-SC,2.37,253.0,-0.99,0.62,10.27,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,0.14,-12.75,14.63,0.02,22283.0,-22250.0,152309.0,132.14,51.0,M-SC,16.22,234.0,-1.00,1.13,32.48,OX40N,NTT,PAINTS
50,MASFIN,398.61,0.20,-2.02,24.57,22.05,23587.0,-1980.0,96000.0,-15.54,56.0,H-SC,7.10,164.0,-0.08,0.71,39.54,XR,ATH,FINANCE
43,ITC,452.00,-0.10,-1.60,12.23,10.44,24086.0,-3193.0,196945.0,-41.15,46.0,X-LC,1.75,5.0,-0.13,1.46,3.88,X40,NTT,FMCG
82,VOLTAS,1530.00,0.92,5.86,13.07,19.69,26529.0,11238.0,202980.0,-4.53,49.0,X-MC,2.57,78.0,0.42,1.51,13.06,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,5.02,-16.70,45.95,21.58,53642.0,-23400.0,116739.0,-57.99,29.0,X-MC,7.08,56.0,-0.44,0.87,5.02,X40N,ATH,IT
72,TATAELXSI,9161.00,2.97,-27.83,82.95,32.03,78920.0,-36693.0,95142.0,-21.91,39.0,H-SC,8.57,148.0,-0.46,0.71,6.57,OX40N,NTT,IT
18,COFFEEDAY,80.00,2.95,-43.35,138.81,35.27,89283.0,-49229.0,64320.0,-55.73,35.0,L-SC,10.75,270.0,-0.55,0.48,57.42,XR,NTT,HOTELS
6,ASIANTILES,137.00,2.89,-15.41,114.77,81.67,91076.0,-14455.0,79355.0,7133.33,49.0,L-SC,14.01,271.0,-0.16,0.59,53.67,XR,NTT,CERAMICS
45,JIOFIN,387.00,2.80,-3.82,29.43,24.48,75676.0,-10225.0,257140.0,-5.47,46.0,X-LC,5.95,36.0,-0.14,1.91,49.05,XY24,BTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,JCHAC,2282.00,-1.77,-36.89,58.44,-0.01,47136.0,-47148.0,80657.0,16400.00,23.0,M-SC,5.00,233.0,-1.00,0.60,3.23,OX40N,NTT,AC
14,CAMPUS,393.00,-1.71,-10.47,48.86,33.28,73396.0,-17559.0,150216.0,-27.85,41.0,M-SC,2.81,222.0,-0.24,1.12,16.43,XY24,NTT,FOOTWEAR
12,BERGEPAINT,680.00,-1.69,-3.84,25.97,21.14,56775.0,-8720.0,218619.0,-10.57,40.0,X-MC,4.18,74.0,-0.15,1.62,23.23,XY24,NTT,PAINTS
67,SONACOMS,806.63,-1.59,-18.42,71.06,39.54,58639.0,-18638.0,82521.0,-36.23,38.0,M-SC,4.69,220.0,-0.32,0.61,16.36,AR,ATH,AUTO
30,HATHWAY,31.40,-1.48,-30.19,148.22,73.29,112499.0,-32820.0,75900.0,780.43,39.0,H-SC,7.50,172.0,-0.29,0.56,2.35,XR,NTT,ENTERTAINMENT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.03,-6.22,121.71,107.91,168514.0,-9189.0,138455.0,-23.79,41.0,M-SC,12.33,216.0,-0.05,1.03,3.23,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.14,-12.75,14.63,0.02,22283.0,-22250.0,152309.0,132.14,51.0,M-SC,16.22,234.0,-1.00,1.13,32.48,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.83,-16.72,20.18,0.09,16937.0,-16845.0,83932.0,94.88,43.0,M-SC,2.37,253.0,-0.99,0.62,10.27,OX40N,NTT,DURABLES
66,SIS,528.00,-0.79,-25.47,62.71,21.26,51891.0,-28284.0,82748.0,1950.26,43.0,H-SC,4.62,166.0,-0.55,0.61,11.80,OX40N,NTT,MISC
47,KANSAINER,340.00,-0.61,-25.49,52.29,13.47,105068.0,-68733.0,200934.0,-69.28,30.0,H-SC,4.18,158.0,-0.65,1.49,2.17,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.48,6.32,62.03,72.27,111748.0,10703.0,180151.0,-8.8,66.0,M-LC,1.93,99.0,0.1,1.34,12.88,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.48,6.32,62.03,72.27,111748.0,10703.0,180151.0,-8.80,66.0,M-LC,1.93,99.0,0.10,1.34,12.88,XR,NTT,IT
38,INDIAMART,4810.62,-0.07,-4.04,115.42,106.72,136604.0,-4982.0,118354.0,-52.54,31.0,H-SC,2.00,139.0,-0.04,0.88,18.39,AR,ATH,MISC
25,FINCABLES,1641.55,-0.03,-6.22,121.71,107.91,168514.0,-9189.0,138455.0,-23.79,41.0,M-SC,12.33,216.0,-0.05,1.03,3.23,OX40N,ATH,CABLES
37,IEX,219.00,2.16,-3.31,53.71,48.63,103846.0,-6609.0,193345.0,-35.77,51.0,H-SC,14.05,136.0,-0.06,1.44,7.87,XR,NTT,MISC
85,ZYDUSLIFE,1286.17,0.00,-2.60,39.26,35.63,80136.0,-5458.0,204116.0,-16.30,38.0,H-MC,3.15,119.0,-0.07,1.52,13.63,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.35,-21.57,89.28,48.45,140402.0,-43246.0,157260.0,-27.18,25.0,X-MC,12.19,64.0,-0.31,1.17,2.39,X40N,ATH,FINANCE
22,DIXON,18931.72,5.02,-16.70,45.95,21.58,53642.0,-23400.0,116739.0,-57.99,29.0,X-MC,7.08,56.0,-0.44,0.87,5.02,X40N,ATH,IT
33,HINDUNILVR,2922.00,0.25,-10.24,26.64,13.67,61469.0,-26323.0,230740.0,-16.94,30.0,X-LC,2.63,14.0,-0.43,1.71,8.93,XY25,NTT,FMCG
75,TMPV,600.00,1.03,-17.78,72.94,42.18,165758.0,-49158.0,227252.0,-26.74,30.0,X-LC,6.04,3.0,-0.30,1.69,1.03,XY24,NTT,AUTO
51,MEDANTA,1486.00,-1.23,-4.58,30.03,24.08,36377.0,-5813.0,121137.0,-12.23,31.0,X-SC,5.42,89.0,-0.16,0.90,14.14,XY24,NTT,HEALTHCARE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.0,2.58,-80.18,529.73,24.79,264383.0,-201947.0,49909.0,-81.09,45.0,X-SC,1.27,86.0,-0.76,0.37,22.01,X40N,NTT,MISC
8,AWL,485.0,0.14,-21.67,95.96,53.50,226813.0,-65389.0,236362.0,-62.10,33.0,X-MC,1.71,58.0,-0.29,1.76,4.98,XY24,NTT,FMCG
43,ITC,452.0,-0.10,-1.60,12.23,10.44,24086.0,-3193.0,196945.0,-41.15,46.0,X-LC,1.75,5.0,-0.13,1.46,3.88,X40,NTT,FMCG
36,ICICIPRULI,790.0,-1.17,5.32,24.35,30.97,45326.0,9405.0,186143.0,-19.43,59.0,X-MC,2.01,75.0,0.21,1.38,18.55,X40,ATH,INSURANCE
3,ACC,3906.0,-0.24,-25.28,119.69,64.15,212809.0,-60151.0,177800.0,-56.31,34.0,X-SC,2.11,82.0,-0.28,1.32,0.00,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.24,-25.28,119.69,64.15,212809.0,-60151.0,177800.0,-56.31,34.0,X-SC,2.11,82.0,-0.28,1.32,0.00,XY24,BTT,CEMENT
54,PGHH,17907.65,-0.13,-6.01,42.32,33.76,79877.0,-12075.0,188745.0,-33.72,33.0,X-MC,5.23,57.0,-0.15,1.40,0.00,X40,ATH,FMCG
31,HAVELLS,2069.16,0.84,-10.26,47.63,32.48,140192.0,-33655.0,294336.0,-14.76,39.0,X-MC,5.49,67.0,-0.24,2.19,0.84,X40,ATH,ELECTRICAL
53,PAGEIND,51605.07,1.02,-9.44,39.21,26.07,58141.0,-15460.0,148280.0,-33.27,33.0,X-MC,12.13,55.0,-0.27,1.10,1.02,X40,ATH,APPARELS
75,TMPV,600.00,1.03,-17.78,72.94,42.18,165758.0,-49158.0,227252.0,-26.74,30.0,X-LC,6.04,3.0,-0.30,1.69,1.03,XY24,NTT,AUTO


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.14,-29.91,103.80,42.84,47294.0,-19443.0,45563.0,-54.27,41.0,X-SC,36.53,83.0,-0.41,0.34,3.08,XY24,NTT,MISC
15,CAMS,4762.00,2.58,-80.18,529.73,24.79,264383.0,-201947.0,49909.0,-81.09,45.0,X-SC,1.27,86.0,-0.76,0.37,22.01,X40N,NTT,MISC
59,RELAXO,1176.00,-1.36,-47.41,189.58,52.31,144739.0,-68813.0,76347.0,-44.05,43.0,X-SC,6.43,91.0,-0.48,0.57,2.19,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.29,-38.07,118.33,35.20,94285.0,-48990.0,79680.0,4.40,34.0,X-SC,15.68,92.0,-0.52,0.59,1.21,X40,NTT,FOOTWEAR
22,DIXON,18931.72,5.02,-16.70,45.95,21.58,53642.0,-23400.0,116739.0,-57.99,29.0,X-MC,7.08,56.0,-0.44,0.87,5.02,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.09,-13.27,37.53,19.28,112608.0,-45908.0,300048.0,-24.27,60.0,X-LC,5.21,1.0,-0.41,2.23,10.92,X40,ATH,IT
41,INFY,2275.00,0.81,8.76,42.39,54.87,144257.0,27422.0,340310.0,-15.29,62.0,X-LC,3.23,2.0,0.19,2.53,17.77,X40,BTT,IT
75,TMPV,600.00,1.03,-17.78,72.94,42.18,165758.0,-49158.0,227252.0,-26.74,30.0,X-LC,6.04,3.0,-0.30,1.69,1.03,XY24,NTT,AUTO
81,VBL,671.64,1.15,-3.36,40.39,35.67,123278.0,-10623.0,305219.0,-14.83,57.0,X-LC,4.58,4.0,-0.09,2.27,10.03,X40N,ATH,FMCG
43,ITC,452.00,-0.10,-1.60,12.23,10.44,24086.0,-3193.0,196945.0,-41.15,46.0,X-LC,1.75,5.0,-0.13,1.46,3.88,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,2.89,-15.41,114.77,81.67,91076.0,-14455.0,79355.0,7133.33,49.0,L-SC,14.01,271.0,-0.16,0.59,53.67,XR,NTT,CERAMICS
50,MASFIN,398.61,0.20,-2.02,24.57,22.05,23587.0,-1980.0,96000.0,-15.54,56.0,H-SC,7.10,164.0,-0.08,0.71,39.54,XR,ATH,FINANCE
13,BSOFT,831.70,0.94,-22.57,94.73,50.79,103170.0,-31743.0,108910.0,-0.66,66.0,H-SC,5.85,151.0,-0.31,0.81,26.81,XR,ATH,IT
39,INDIGOPNTS,1408.00,0.14,-12.75,14.63,0.02,22283.0,-22250.0,152309.0,132.14,51.0,M-SC,16.22,234.0,-1.00,1.13,32.48,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.93,-2.89,24.54,20.95,60000.0,-7269.0,244499.0,-3.87,46.0,X-LC,18.65,31.0,-0.12,1.82,31.72,X40,ATH,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,0.94,-22.57,94.73,50.79,103170.0,-31743.0,108910.0,-0.66,66.0,H-SC,5.85,151.0,-0.31,0.81,26.81,XR,ATH,IT
56,RAJESHEXPO,518.00,1.12,-62.84,169.29,0.07,86296.0,-86202.0,50975.0,1768.69,58.0,L-SC,2.39,268.0,-1.00,0.38,26.78,OX40N,NTT,JEWELLERY
84,WIPRO,420.00,0.48,6.32,62.03,72.27,111748.0,10703.0,180151.0,-8.80,66.0,M-LC,1.93,99.0,0.10,1.34,12.88,XR,NTT,IT
32,HCLTECH,1922.01,0.35,8.49,14.97,24.73,39292.0,20549.0,262473.0,13.18,64.0,X-LC,6.21,7.0,0.52,1.95,25.02,X40,ATH,IT
27,GLAXO,3466.20,1.04,5.18,32.29,39.15,67685.0,10332.0,209616.0,-15.47,60.0,X-MC,11.24,60.0,0.15,1.56,35.69,X40,ATH,PHARMA


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.79
1,25,45.77
2,50,76.72


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.04
MC    30.29
LC    24.69
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.28
X40      22.44
X40N     10.85
XR       10.15
XY25      9.40
AR        9.35
OX40N     7.78
SR        1.02
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.54
X-MC    23.52
X-LC    21.10
M-SC    12.11
X-SC     6.94
H-MC     5.01
L-SC     1.45
M-MC     1.41
M-LC     1.34
H-LC     1.19
L-LC     1.06
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.38,-7.55,42.33
IT,12.79,-17.14,80.46
FINANCE,9.59,-20.09,70.83
ELECTRICAL,6.25,-11.61,52.49
MISC,6.11,-58.04,122.78
PAINTS,6.06,-13.10,29.90
INSURANCE,4.60,-2.03,37.98
PHARMA,4.12,-0.91,33.21
AUTO,3.40,-25.64,79.51


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3229193.0,21
AR,1312780.0,10
XR,1295460.0,13
X40,1018511.0,14
X40N,988703.0,9
OX40N,744373.0,10
XY25,376881.0,6
SR,282775.0,2
MH,83360.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3671670.0,25
M-SC,1429195.0,15
X-MC,1317663.0,16
X-SC,985803.0,8
X-LC,922477.0,11
H-MC,404782.0,3
L-SC,266655.0,3
M-LC,111748.0,1
H-LC,73755.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1284918.0      6
           AR         900015.0      5
M-SC       XY24       813758.0      6
H-SC       XR         774323.0      7
X-SC       X40N       507268.0      3
X-MC       X40        456213.0      7
           XY24       397263.0      3
X-LC       X40        380243.0      5
H-SC       OX40N      346279.0      4
M-SC       OX40N      311798.0      5
X-SC       XY24       296480.0      3
H-SC       SR         282775.0      2
X-MC       X40N       280154.0      4
X-LC       XY24       241434.0      2
H-MC       AR         209442.0      2
X-LC       X40N       201281.0      2
H-MC       XY24       195340.0      1
X-MC       XY25       184033.0      2
X-SC       X40        182055.0      2
L-SC       XR         180359.0      2
M-SC       XR         174071.0      2
           AR         129568.0      2
M-LC       XR         111748.0      1
X-LC       XY25        99519.0      2
L-SC       OX40N       86296.0      1
H-SC       MH          83360.0      1
H-LC       AR          73755.0      1
M-MC       XY25        56009.0      1
L-MC       XR          54959.0      1
L-LC       XY25        37320.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
